# Project Akhir

## Preprocessing data

In [9]:
#import the package/Library
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle

In [10]:
# mengakses data 
dataset_url = "https://raw.githubusercontent.com/triasmidwi/dataset/main/milknew.csv"
# membaca data
df = pd.read_csv(dataset_url)
#menampilkan 5 data pertama
df.head()

,pH,Temprature,Taste,Odor,Fat,Turbidity,Colour,Grade
0,6.6,35,1,0,1,0,254,high
1,6.6,36,0,1,0,1,253,high
2,8.5,70,1,1,1,1,246,low
3,9.5,34,1,1,0,1,255,low
4,6.6,37,0,0,0,0,255,medium


In [11]:
#mengecek jumlah baris dan jumlah kolom
df.shape

(1059, 8)

In [12]:
#membuat DataFrame kecuali kolom yang berisi target
X = df.drop(columns=['Grade'])
#mengecek apakah kolom targer sudah berhasil dihilangkan
X.head()

,pH,Temprature,Taste,Odor,Fat,Turbidity,Colour
0,6.6,35,1,0,1,0,254
1,6.6,36,0,1,0,1,253
2,8.5,70,1,1,1,1,246
3,9.5,34,1,1,0,1,255
4,6.6,37,0,0,0,0,255


In [13]:
#memisahkan kolom target dan dimasukkan ke dalam variable y
y = df['Grade'].values
#menampilkan isi kolom target dari data 0-4
y[0:5]

array(['high', 'high', 'low', 'low', 'medium'], dtype=object)

In [14]:
#melakukan split dataset ke dalam bentuk data train dan data test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

In [15]:
classifier = RandomForestClassifier()

In [16]:
classifier.fit(X_train, y_train)

RandomForestClassifier()

In [17]:
RandomForestClassifier()

RandomForestClassifier()

In [18]:
print(f"\nStacking classifier training Accuracy: {classifier.score(X_train, y_train):0.2f}")
print(f"Stacking classifier test Accuracy: {classifier.score(X_test, y_test):0.2f}")


Stacking classifier training Accuracy: 1.00
Stacking classifier test Accuracy: 1.00


In [ ]:
pickle.dump(classifier, open("model.pkl", "wb"))